In [2]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.applications.resnet50 import decode_predictions
from tensorflow.keras.applications.resnet50 import preprocess_input
import tensorflow as tf
import numpy as np
import cv2

In [3]:
print(tf.test.is_gpu_available())
print(tf.config.list_physical_devices('GPU'))

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
EPS = .001
LR = 0.1
sccLoss = SparseCategoricalCrossentropy()
optimizer = Adam(learning_rate=LR)

In [28]:
def preprocess_image(image):
	# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	image = cv2.resize(image, (224, 224))
	image = np.expand_dims(image, axis=0)
	return image

In [4]:
def clip_eps(tensor, eps):
	return tf.clip_by_value(tensor, clip_value_min=-eps,
		clip_value_max=eps)

In [5]:
def generate_adversaries(model, baseImage, delta, classIdx, steps=50):
	for step in range(0, steps):
		# track gradients
		with tf.GradientTape() as tape:
			# track perturbation vector for gradient updates
			tape.watch(delta)
			# add our perturbation vector to the base image 
			adversary = preprocess_input(baseImage + delta)
			# run perturbated image through model and calculate the loss with respect to the class index
			predictions = model(adversary, training=False)
			loss = -sccLoss(tf.convert_to_tensor([classIdx]),
				predictions)
			# log the loss
			if step % 5 == 0:
				print("step: {}, loss: {}...".format(step,
					loss.numpy()))
		# calculate the gradients of loss with respect to the
		# perturbation vector
		gradients = tape.gradient(loss, delta)
		# update the weights, clip the perturbation vector, and
		# update its value
		optimizer.apply_gradients([(gradients, delta)])
		delta.assign_add(clip_eps(delta, eps=EPS))
	# return the perturbation vector
	return delta

In [31]:
image_path = tf.keras.utils.get_file('YellowLabradorLooking_new.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg')
# image_raw = tf.io.read_file(image_path)
# image = tf.image.decode_image(image_raw)
image = cv2.imread(image_path)
image = preprocess_image(image)

model = ResNet50(weights="imagenet")

baseImage = tf.constant(image, dtype=tf.float32)
cv2.imwrite("temppics\\input.png", baseImage.numpy().squeeze())

delta = tf.Variable(tf.zeros_like(baseImage), trainable=True)
print("[INFO] generating perturbation...")
deltaUpdated = generate_adversaries(model, baseImage, delta, 208)

print("[INFO] creating adversarial example...")
cv2.imwrite("temppics\\change.png", (deltaUpdated * 50).numpy().squeeze())
adverImage = (baseImage + deltaUpdated).numpy().squeeze()
# adverImage = np.clip(adverImage, 0, 255).astype("uint8")
# adverImage = cv2.cvtColor(adverImage, cv2.COLOR_RGB2BGR)
cv2.imwrite("temppics\\output.png", adverImage)

# run inference with this adversarial example
print("[INFO] running inference on the adversarial example...")
preprocessedImage = preprocess_input(baseImage + deltaUpdated)
predictions = model.predict(preprocessedImage)
predictions = decode_predictions(predictions, top=3)[0]
print(predictions)
label = predictions[0][1]
confidence = predictions[0][2] * 100
print("[INFO] label: {} confidence: {:.2f}%".format(label,
	confidence))

[INFO] generating perturbation...
step: 0, loss: -2.2798099517822266...
step: 5, loss: -10.055440902709961...
step: 10, loss: -19.210041046142578...
step: 15, loss: -28.864030838012695...
step: 20, loss: -37.69005584716797...
step: 25, loss: -46.55704879760742...
step: 30, loss: -55.62173843383789...
step: 35, loss: -65.60511016845703...
step: 40, loss: -75.1745834350586...
step: 45, loss: -84.1983871459961...
[INFO] creating adversarial example...
[INFO] running inference on the adversarial example...
1/1 [==============================] - 1s 582ms/step
[('n02091831', 'Saluki', 1.0), ('n02091134', 'whippet', 4.0236542e-12), ('n02090622', 'borzoi', 6.5043016e-14)]
[INFO] label: Saluki confidence: 100.00%
